## Reuseable Sentinel-1 (SAR) functions and templates


In [1]:
# pre-install and import all the libraries in one go
import numpy as np
import ee   
import numpy as np
import os
import time
import pandas as pd
import geemap
import geemap.colormaps as cm

In [2]:
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

## Sentinel-1 functions

In [4]:
#glcm function for Sentinel-1 texture
def glcmMap (i, bands):
  imageVH = i.select(bands)
  imageVH = ee.Image(10).pow(imageVH.divide(10)).rename(bands)
  square = ee.Kernel.square(3)
  result_image = imageVH.unitScale(0,1).multiply(255).toByte().glcmTexture(3, square)
  return i.addBands(result_image)

def FocalMeanMap (i, bands, radius):
  image = i.select(bands)
  renamed_bands = []
  for band in bands:
    renamed_bands.append(band + '_Focal')
  focal_image = image.focalMean(radius, 'circle', 'meters').rename(renamed_bands)
  return i.addBands(focal_image)

def CalcGamma(img):
	img = ee.Image(img)
	img_vv = ee.Image(img).select('VV')
	img_vh = ee.Image(img).select('VH')
	img_vv = img_vv.subtract(((img.select('angle').multiply(3.1415927).divide(180)).cos()).log10().multiply(10))
	img_vh = img_vh.subtract(((img.select('angle').multiply(3.1415927).divide(180)).cos()).log10().multiply(10))
	img_all = img_vv.addBands(img_vh).copyProperties(img).copyProperties(img, ['system:time_start','system:time_end'])
	return img_all;

def S1FilterAngles( im_sar ):
  im_sar = ee.Image( im_sar )
  sar_min_angle = 0
  sar_max_angle = 50.9
  im_sar_min = im_sar.select( 'angle' ).gt( sar_min_angle )
  im_sar_max = im_sar.select( 'angle' ).lt( sar_max_angle )
  im_sar_centre = im_sar_min.And( im_sar_max )
  im_mask = im_sar.mask().And( im_sar_centre )
  return im_sar.updateMask( im_mask )

## Get Sentinel-1 data

In [7]:
#Sentinel-1
geom = ee.Geometry.BBox(-121.775764, 37.416014, -121.769112, 37.420122)
s1_ic = ee.ImageCollection("COPERNICUS/S1_GRD").filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('system:band_names',['VV','VH','angle'])).filterBounds(geom)
s1_ic = s1_ic.filterDate('2022-04-01', '2022-08-01').map(S1FilterAngles).map(CalcGamma)
s1_im = s1_ic.mean()
s1_im = s1_im.addBands(s1_im.select('VV').subtract(s1_im.select('VH')).rename('VVmVH'))
s1_im = s1_im.select(['VV', 'VH', 'VVmVH'])


s1Map = geemap.Map()
s1Map.centerObject(geom, 11)
s1Map.addLayer(s1_im, {'min':-20, 'max':0, 'bands': ['VV']}, '2022 Sentinel-2')
s1Map.addLayer(geom, {"color": '#00000000'}, 'geometry')
s1Map

Map(center=[37.418067998714086, -121.77243799997117], controls=(WidgetControl(options=['position', 'transparen…